In [6]:
# Sample 1 - Reading a CSV file and inserting data into a table
import csv
import psycopg2
import pandas as pd
import json

inputFile = 'Resource/gdp-per-capita-penn-world-table.csv';

In [7]:
with open(inputFile, 'r') as file:
  csvreader = csv.reader(file)
  print('before the loop');
  recordCount = 0;
  for row in csvreader:
    print(recordCount,row[0],row[1],row[2],row[3]);
    
    recordCount += 1;
print('after the loop');

before the loop
0 Entity Code Year GDP per capita (output, multiple price benchmarks)
1 Albania ALB 1970 3114.0884
2 Albania ALB 1971 3159.8088
3 Albania ALB 1972 3214.6665
4 Albania ALB 1973 3267.8481
5 Albania ALB 1974 3330.0708
6 Albania ALB 1975 3385.273
7 Albania ALB 1976 3431.9856
8 Albania ALB 1977 3501.2654
9 Albania ALB 1978 3579.4128
10 Albania ALB 1979 3598.447
11 Albania ALB 1980 3714.541
12 Albania ALB 1981 3786.318
13 Albania ALB 1982 3724.358
14 Albania ALB 1983 3775.8096
15 Albania ALB 1984 3738.4922
16 Albania ALB 1985 3689.6462
17 Albania ALB 1986 3781.372
18 Albania ALB 1987 3753.8347
19 Albania ALB 1988 3589.6863
20 Albania ALB 1989 3816.0518
21 Albania ALB 1990 3681.2083
22 Albania ALB 1991 3299.2512
23 Albania ALB 1992 3009.153
24 Albania ALB 1993 3512.3381
25 Albania ALB 1994 4027.3682
26 Albania ALB 1995 4391.234
27 Albania ALB 1996 4862.4824
28 Albania ALB 1997 4539.818
29 Albania ALB 1998 4993.022
30 Albania ALB 1999 5292.0273
31 Albania ALB 2000 5315.43
32 Al

In [ ]:
# Another way to read a CSV and keep it for further processing 
# import pandas as pd
# data = pd.read_csv(inputFile)
# data

In [8]:

# Use your own dabaase name, user and password
conn = psycopg2.connect(database="worldfacts",
						user='postgres', password='top2gun6',
						host='127.0.0.1', port='5432'
)

conn.autocommit = True
cursor          = conn.cursor()
#open the csv file using python standard file I/O
#copy file into the table just created 
with open(inputFile, 'r') as f:
    next(f); # Skip the header row.
    #f , <database name>, Comma-Seperated
    cursor.copy_from(f, 'gdp', sep=',');
    #Commit Changes
    conn.commit();

UniqueViolation: duplicate key value violates unique constraint "country_code"
DETAIL:  Key (country_code, year)=(ALB, 1970) already exists.
CONTEXT:  COPY gdp, line 1


In [9]:
sql_select_Query = "select * from gdp"
cursor = conn.cursor()
cursor.execute(sql_select_Query)
# get all records
records = cursor.fetchall()
print("Total number of rows in table: ", cursor.rowcount)

print("\nPrinting each row")
for row in records:
    print("country", row[0], "country code", row[1], "year", row[2],"gdp", row[3])


Total number of rows in table:  10399

Printing each row
country Albania country code ALB year 1970 gdp 3114.0884
country Albania country code ALB year 1971 gdp 3159.8088
country Albania country code ALB year 1972 gdp 3214.6665
country Albania country code ALB year 1973 gdp 3267.8481
country Albania country code ALB year 1974 gdp 3330.0708
country Albania country code ALB year 1975 gdp 3385.2730
country Albania country code ALB year 1976 gdp 3431.9856
country Albania country code ALB year 1977 gdp 3501.2654
country Albania country code ALB year 1978 gdp 3579.4128
country Albania country code ALB year 1979 gdp 3598.4470
country Albania country code ALB year 1980 gdp 3714.5410
country Albania country code ALB year 1981 gdp 3786.3180
country Albania country code ALB year 1982 gdp 3724.3580
country Albania country code ALB year 1983 gdp 3775.8096
country Albania country code ALB year 1984 gdp 3738.4922
country Albania country code ALB year 1985 gdp 3689.6462
country Albania country code AL

In [10]:
cursor.execute(sql_select_Query)
table_rows = cursor.fetchall()
df = pd.DataFrame(table_rows)
df.columns = ['Country', 'Country_Code','Year', 'GDP']
print(df.head)
result = df.to_json(orient="records")
parsed = json.loads(result)
json_object = json.dumps(parsed, indent=4)
with open("Resource/gdp.json", "w") as outfile:
    outfile.write(json_object)

<bound method NDFrame.head of         Country Country_Code  Year        GDP
0       Albania          ALB  1970  3114.0884
1       Albania          ALB  1971  3159.8088
2       Albania          ALB  1972  3214.6665
3       Albania          ALB  1973  3267.8481
4       Albania          ALB  1974  3330.0708
...         ...          ...   ...        ...
10394  Zimbabwe          ZWE  2015  2880.9058
10395  Zimbabwe          ZWE  2016  2919.6170
10396  Zimbabwe          ZWE  2017  3112.8750
10397  Zimbabwe          ZWE  2018  3007.2370
10398  Zimbabwe          ZWE  2019  2787.6590

[10399 rows x 4 columns]>


In [ ]:
conn.commit();
conn.close();